In [1]:
import os
from topic_gen.input.inject_data import InjectData
import pandas as pd
from dotenv import load_dotenv

load_dotenv()

print(os.getenv('DB_DATABASE'))


inject = InjectData()

user_id = "8c525b6b-27c9-4379-a454-2c3b3a781124"

df = inject.query_bookmarks(user_id)

print(df)

tlwl-dev
                               bookmark_id  \
0     a4f37ecc-172a-48e4-8570-8d059398d77d   
1     a4f37ecc-172a-48e4-8570-8d059398d77d   
2     a4f37ecc-172a-48e4-8570-8d059398d77d   
3     06ba62ab-870f-432a-87a6-a5be07f7b2e6   
4     06ba62ab-870f-432a-87a6-a5be07f7b2e6   
...                                    ...   
2132  d8ae06ab-0b62-4115-86c8-7153834e142a   
2133  d8ae06ab-0b62-4115-86c8-7153834e142a   
2134  d8ae06ab-0b62-4115-86c8-7153834e142a   
2135  d8ae06ab-0b62-4115-86c8-7153834e142a   
2136  d8ae06ab-0b62-4115-86c8-7153834e142a   

                                                    url  \
0                https://www.latent.space/p/2025-papers   
1                https://www.latent.space/p/2025-papers   
2                https://www.latent.space/p/2025-papers   
3     https://www.interconnects.ai/p/papers-im-readi...   
4     https://www.interconnects.ai/p/papers-im-readi...   
...                                                 ...   
2132                   ht

In [2]:
grouped_df = (
    df.groupby(["tag_id", "key", "name"]).apply(lambda g: g[[
        "bookmark_id", "title", "description", "language", "created_at", "updated_at",
    ]].to_dict(orient="records"))
    .reset_index(name="bookmarks")
)

print(grouped_df)

                                   tag_id          key         name  \
0    00608d78-5556-4e9a-8b0c-ce2796f9f13d         qwen         qwen   
1    006adf0c-c6bc-44b3-90f6-cc9f524a90e0  challenging  challenging   
2    03b8761c-7c70-42d5-bf1a-e04aa63c36b5      testing      testing   
3    07db4626-c83f-4ac5-86d4-e49ab2af51a0      scaling      scaling   
4    07e2bc25-09ac-4491-bd68-b52bbbf29059    detection    detection   
..                                    ...          ...          ...   
166  f829442a-a48f-4b74-86dc-2241ddcdbe28      metrics      metrics   
167  f9aaf4e8-18e2-4f48-af22-4a044c4def8d      pixtral      pixtral   
168  fab1bd45-c87f-4267-aa29-65a4ceabb690  fine-tuning  fine tuning   
169  fe0fcf88-552b-4479-b1a1-6838d08eb490       images       images   
170  fee44b01-a2dd-4a65-bb39-51999613b207      dataset      dataset   

                                             bookmarks  
0    [{'bookmark_id': 'c6c0d547-a5de-4d68-9a48-c2f0...  
1    [{'bookmark_id': 'c23de130-b

C:\Users\pasan\AppData\Local\Temp\ipykernel_67408\3638810674.py:2: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df.groupby(["tag_id", "key", "name"]).apply(lambda g: g[[
